#  Export Example
Export a grid in PowerFactory to *.json files applying the 'Power System Data Model' (PSDM)

Instructions:
- Import the provided PowerFactory example (examples/grids/*.pfd) in PowerFactory (if not already done)
- Close the PowerFactory Application
- Run this example
    - initiate the compatible Interface
    - if necessary specify additional attributes to be exported
    - Start the exporter in this process (via A)) or in a dedicated Python process (via B))

The exporter generates for each study case three JSON files: topology, topology_case, steadystate_case.
Here, only active grids will be exported.

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import logging
import pathlib
from collections.abc import Sequence

from psdm.steadystate_case.case import Case as SteadystateCase
from psdm.topology.topology import Topology
from psdm.topology_case.case import Case as TopologyCase

### Select an Interface that is compatible to your PowerFactory version  
- Import the PowerFactoryExporter (in this example: that is compatible with PowerFactory in version 2024)
- Specify additional PowerFactory configuration
- Name the PowerFactory project

In [ ]:
from powerfactory_tools.versions.pf2024 import PowerFactoryExporter
from powerfactory_tools.versions.pf2024.interface import ValidPythonVersion
from powerfactory_tools.versions.pf2024.types import PFClassId

# PowerFactory configuration
PF_SERVICE_PACK = 2  # mandatory
PF_USER_PROFILE = ""  # specification may be necessary
PF_PYTHON_VERSION = ValidPythonVersion.VERSION_3_12  # python version of local code environment must match the python version of PowerFactory API

# Consider to use raw strings to avoid misinterpretation of special characters, e.g. r"dir\New Project" or r"dir\1-HV-grid".
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\project_name"
STUDY_CASES = ["3_Bus", "Base", "Outage", "Industry_Park"]  # list of study cases to be exported
EXPORT_PATH = pathlib.Path("export")

### [Optional] Specify additional attributes to be exported

- In case one wants to export additional attribute data for elements of specified types, one can use the following dictionary.
- The dictionary key is the PowerFactory class ID of the element type, and the value is a list of attribute names.
- The attribute names can be nested, e.g. generator.pQlimType returns a PowerFactory DataObject for which attributes can also be requested: {"pQlimType": {"cap_Ppu", "cap_Qmnpu"}}.

In [ ]:
element_specific_attrs: dict[PFClassId, Sequence[str | dict]] = {
    PFClassId.LINE: ["nlnum", "fline", "typ_id"],  # list of attribute names for elements of type Line
    PFClassId.GENERATOR: [
        "loc_name",
        {"pQlimType": ["cap_Ppu", "cap_Qmnpu"]},  # list of (nested) attribute names for elements of type Generator
    ],
}

## A) Export using an exporter instance
- As the Exporter instance is started within Python process of this Jupyter Notebook, the PowerFactory application is blocked as long as this Jupyter Notebook runs.
- To start PowerFactory again, this Jupyter Notebook has to be terminated/restarted.

In [ ]:
with PowerFactoryExporter(
    project_name=PROJECT_NAME,
    powerfactory_service_pack=PF_SERVICE_PACK,
    powerfactory_user_profile=PF_USER_PROFILE,
    python_version=PF_PYTHON_VERSION,
    logging_level=logging.INFO,
    element_specific_attrs=element_specific_attrs,
) as exporter:
    _project_name = PROJECT_NAME.split("\\")

    # Export each study case of the specified project and save it in a separate folder in the specified export path.
    exporter.export(
        export_path=EXPORT_PATH / pathlib.Path(_project_name[-1]),
        study_case_names=STUDY_CASES,
    )

    # As alternative, export just the study case that is active at the moment
    # exporter.export(export_path=EXPORT_PATH)  # noqa: ERA001

## B) Export using the export function running in a new process with default parameters
- Starts Exporter in a separate process using multihtreading.
- As the process is terminated after export is finished, the PowerFactory application is no longer blocked by the Python process.

In [ ]:
from powerfactory_tools.versions.pf2024 import export_powerfactory_data

ext_log_file_path = pathlib.Path(r"export\logfile.txt")
_project_name = PROJECT_NAME.split("\\")
# As the export function is executed in a process that is terminated after execution, the PowerFactory API is also closed.
export_powerfactory_data(
    project_name=PROJECT_NAME,
    powerfactory_service_pack=PF_SERVICE_PACK,
    powerfactory_user_profile=PF_USER_PROFILE,
    python_version=PF_PYTHON_VERSION,
    export_path=EXPORT_PATH / pathlib.Path(_project_name[-1]),
    study_case_names=STUDY_CASES,
    logging_level=logging.INFO,
    log_file_path=ext_log_file_path,
)

## [Optional] Display JSON files and PSDM schema
### Display example JSON files for study case "Industry_Park"

In [ ]:
t = Topology.from_file(EXPORT_PATH / pathlib.Path("Industry_Park") / "Industry_Park_HV_9_Bus_topology.json")
t_s = t.model_dump()
print(json.dumps(t_s, sort_keys=True, default=str, indent=2))  # noqa: T201

In [ ]:
tc = TopologyCase.from_file(EXPORT_PATH / pathlib.Path("Industry_Park") / "Industry_Park_HV_9_Bus_topology_case.json")
tc_s = tc.model_dump()
print(json.dumps(tc_s, sort_keys=True, default=str, indent=2))  # noqa: T201

In [ ]:
ssc = SteadystateCase.from_file(
    EXPORT_PATH / pathlib.Path("Industry_Park") / "Industry_Park_HV_9_Bus_steadystate_case.json",
)
ssc_s = ssc.model_dump()
print(json.dumps(ssc_s, sort_keys=True, default=str, indent=2))  # noqa: T201

### Display raw JSON schema of Power System Data Model

In [ ]:
t_s = Topology.model_json_schema()
print(json.dumps(t_s, sort_keys=True, default=str, indent=2))  # noqa: T201

In [ ]:
tc_s = TopologyCase.model_json_schema()
print(json.dumps(tc_s, sort_keys=True, default=str, indent=2))  # noqa: T201

In [ ]:
ssc_s = SteadystateCase.model_json_schema()
print(json.dumps(ssc_s, sort_keys=True, default=str, indent=2))  # noqa: T201